In [25]:
import math
import numpy as np
import pandas as pd
from simulation_util import *

In [26]:
## scenario switch
case=3 # 2 means no trade in goods; 3 means trade in both energy and goods
logit = 0 # 1 means logit estimations of supply elasticity; 0 means fixed elasticities

## parameter values
theta = 4             # scopevec for comparative advantage
sigma = 1      # elasticity of demand for each individual manufactured good j at Home
sigmastar = 1  # elasticity of demand for each individual manufactured good j at Foreign

#TO CHANGE RHO: rho and alpha are defined in simulation_util, which can be changed directly.

# beta gamma not used in the code unless logit indicated
beta=1.892412
gamma=0.807998928

In [27]:
# elasticity of supply of energy
epsilonSstar1 = 0.5
epsilonS1 = 0.5

# baseline, no renewable energy
epsilonSvec = [(epsilonS1, 1, 1)]
epsilonSstarvec = [(epsilonSstar1, 1, 1)]
assert(sum(k for i,j,k in epsilonSvec) == 1)
assert(sum(k for i,j,k in epsilonSstarvec) == 1)

In [28]:
## import BAU values (seven regional scenarios in the order of US, EU, OECD, World, China, OECD plus China)
if case==2:
    df = pd.read_csv("../../raw_data/BaselineCarbon_2015_noTradeinGoods.csv",index_col=['region_scenario','regionbase'],header='infer')
elif case==3:
    df = pd.read_csv("../../raw_data/BaselineCarbon_2015.csv", index_col=['regionbase'],header='infer')
df['jxbar']=df['CeFH']/(df['CeFH'] + df['CeFF'])
df['jmbar']=df['CeHH']/(df['CeHH'] + df['CeHF'])

# Region scenario = World is not stable, so dropping it is recommended
df = df[df['region_scenario'] != 4]
## choose which regional scenario to run (runs all if not executed)
#df=df.drop([2,3,4,5,6,7])  
df = df[df['region_scenario'] == 3]
df

,region_scenario,Qe,Qestar,Qeworld,CeHH,CeHF,CeFH,CeFF,Ce,Cestar,Ge,Gestar,Ceworld,Geworld,jxbar,jmbar
regionbase,,,,,,,,,,,,,,,,
OECD37 as Home,3,8.62549,23.6505,32.27599,11.29367,2.48754,0.91057,17.58421,13.78121,18.49478,12.20424,20.07175,32.27599,32.27599,0.049234,0.819498


In [30]:
# choose desired tax scenarios, base = 0 for Unilateral and 1 otherwise
# base is used as an indicator to simplify the code
#tax_scenario= pd.DataFrame({'tax_sce': ['Unilateral','global','purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid'], 'Base':[0,1,1,1,1,1,1,1,1]},index=[1,2, 3, 4, 5, 6, 7, 8,9])
tax_scenario= pd.DataFrame({'tax_sce': ['Unilateral','purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid'], 'Base':[0,1,1,1,1,1,1,1]},index=[1,2, 3, 4, 5, 6, 7, 8])

In [31]:
# given tax scenario, iterate over chosen values of varphilist for all countries in df.
# returns a pandas df that contains simulation results for all countries given the tax scenario
def iterate_tax(tax_scenario,df,theta, sigma, sigmastar, epsilonSvec, epsilonSstarvec, beta, gamma, logit):
    paralist = (theta, sigma, sigmastar, epsilonSvec, epsilonSstarvec, beta, gamma, logit)
    #varphilist = np.arange(0,3,0.1)
    #varphilist = np.arange (0,1,0.2)
    varphilist = [1,2, 2.1]
    output=[]
    prevtb = []
    for varphi in varphilist:    
        tax_df=df.apply(iterate_region, axis=1, raw=False, args=(prevtb, varphi, tax_scenario, paralist))
        prevtb = tax_df[['region_scenario','tb', 'pe', 'te', 'prop']]
        output.append(tax_df)
        print(varphi)
        
    output = pd.concat(output, axis=0, join='outer',  ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False,copy=True)
    output.reset_index(level=0, inplace=True)
    output = output.sort_values(by=['region_scenario','varphi'])
    
    # if extraction tax too large home stops extracting
    if tax_scenario['tax_sce']=='purete' or tax_scenario['tax_sce']=='EP_hybrid' or tax_scenario['tax_sce'] == 'Unilateral':
        output.te[output.Qe_prime==0]=output.pe+output.tb
    print(tax_scenario['tax_sce'])
    return output

In [32]:
# given the country df of initial baseline values (Qe, Qe* etc), previous tb values, varphi and tax_scenario
# returns df of calculated simulation results for the given country
def iterate_region(df, prevtb, varphi, tax_scenario, paralist):
    # initial guess if previous value isn't available
    pe = 1
    te = 0.5
    tb_mat = [0,0.5]
    
    # use previous vector of solution if available
    if (type(prevtb) != list):
        curr_region = prevtb[prevtb['region_scenario'] == df['region_scenario']]
        vals = curr_region.values
        tb_mat = [vals[0][1], vals[0][4]]
        pe = vals[0][2]
        te = vals[0][3]
    tax_temp = tax_eq(pe, te, tb_mat, df, tax_scenario, varphi, paralist)
    tax_temp.opt_tax()
    ret = tax_temp.retrieve()
    return ret

In [ ]:
# results df contains all simulation results given tax_scenario and df.
results_df = tax_scenario.apply(iterate_tax, axis=1, args=(df,theta, sigma, sigmastar, epsilonSvec, epsilonSstarvec, beta, gamma, logit))

In [10]:
# concat all results into 1 nice df
output_list=[]
for i in range(1,len(tax_scenario)+1):
    output_list.append(results_df.loc[i])
Outcomes = pd.concat(output_list, axis=0, join='outer', ignore_index=False, keys=tax_scenario['tax_sce'], levels=None, verify_integrity=False,copy=True)
Outcomes.reset_index(level=0, inplace=True)
Outcomes

,tax_sce,regionbase,varphi,pe,tb,prop,te,jxbar_prime,jmbar_prime,j0_prime,...,delta_Vg,delta_Vgstar,welfare,welfare_noexternality,Qe1_prime,Qe1star_prime,Qe2_prime,Qe2star_prime,region_scenario,conv
0,PC_hybrid,OECD37 as Home,1.0,0.901578,0.434394,0.459677,0.434394,0.043127,0.819498,0.039292,...,-3.991858,1.718870,0.985098,-0.788471,8.190030,22.456498,0.0,0.0,3.0,1.0
1,PC_hybrid,OECD37 as Home,2.0,0.849256,0.823914,0.531648,0.823914,0.039718,0.819498,0.033325,...,-7.093466,2.714948,3.291538,-2.220351,7.948830,21.795144,0.0,0.0,3.0,1.0
2,PC_hybrid,OECD37 as Home,2.1,0.845357,0.861513,0.537821,0.861513,0.039458,0.819498,0.032854,...,-7.368281,2.791730,3.570882,-2.372853,7.930561,21.745053,0.0,0.0,3.0,1.0


In [13]:
# optionally save results
if epsilonS == 0.5 and epsilonSstar == 0.5:
    Outcomes.to_csv('output/output_case{0}.csv'.format(case), header=True) 
# high foreign elasticity of supply
elif epsilonS == 0.5 and epsilonSstar == 2:
    Outcomes.to_csv('output/output_case{0}_D_2.csv'.format(case), header=True) 
# high home elasiticty of supply
elif epsilonS == 2 and epsilonSstar == 0.5:
    Outcomes.to_csv('output/output_case{0}_2_D.csv'.format(case), header=True) 